# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [5]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


interactive(children=(Select(description='x', options=('scratch', 'proj'), value='scratch'), Output()), _dom_c…

In [6]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20200911_083711_bsize64_spec_test_128_nospec', '2020…

In [9]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201211_201621_bsize256_scale0.1_deterministic_on


In [10]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [11]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [12]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [13]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [14]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,12.27440,1.221220,1.210230,8.58094,12.6203,8.49474,0.621437,0.085051,6.626780,11.6148,641.644,6.09499,"0.104588s mean, 5.6214s max, 0.0288343s min, ...","0.0621501s mean, 0.774084s max, 0.0229897s mi..."
1,6.10809,0.478141,0.431972,4.03024,11.6773,8.12840,0.320655,0.163535,6.489070,11.5514,619.514,5.94449,"0.0940542s mean, 1.3606s max, 0.0287173s min,...","0.0606144s mean, 0.522929s max, 0.0172682s mi..."
2,5.97421,0.378753,0.345494,4.08581,11.6415,19.80670,8.337880,0.117657,10.197700,11.5348,600.942,5.53722,"0.0995262s mean, 2.29193s max, 0.0292394s min...","0.0564595s mean, 0.637471s max, 0.0173649s mi..."
3,5.64253,0.342760,0.334495,3.80658,11.5869,6.74762,1.183040,0.147876,4.265330,11.5137,572.851,5.89908,"0.0954179s mean, 1.06379s max, 0.0287056s min...","0.0601471s mean, 0.800216s max, 0.0177105s mi..."
4,4.88136,0.405950,0.397718,2.92627,11.5142,3.61864,0.075061,2.097930,0.281822,11.6383,614.539,6.12168,"0.0952203s mean, 0.877604s max, 0.0284455s mi...","0.0624112s mean, 0.644383s max, 0.0172461s mi..."


In [15]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))


interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [16]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [17]:
print(df.shape)
df.tail()


(879, 22)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
874,9,8740,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.031404,0.317530,0.000059,0.348993,0.000183,10.314849,...,28.776510,256,5,1,"[0.6929152185048898, 0.027321077391300987, 0.0...","[0.00017331099498061939, 0.0005982878666225563...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[103427.81298539415, 68127.01590843867, 48345....","[50257.18909787113, 25072.91667405983, 17516.8...",9-8740
875,9,8750,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.012246,0.028393,0.000000,0.040639,0.000074,5.436077,...,31.581340,256,2,0,"[0.6922641759095137, 0.029784393269770608, 0.0...","[0.00017724956485106933, 0.0006237190623132054...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[112794.90202955157, 68843.27591388485, 46551....","[62510.4627772183, 28116.220665159013, 15715.2...",9-8750
876,9,8760,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.061069,0.019384,0.000012,0.080464,0.000349,3.262326,...,28.598383,256,1,0,"[0.6940128808210407, 0.024989891018033714, 0.0...","[0.00012041020779041414, 0.0004606986336518537...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[82263.00930202566, 52483.277009225174, 39941....","[40701.81345575592, 17920.948487060254, 13194....",9-8760
877,9,8770,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.125672,0.004806,0.000000,0.130478,0.000869,0.746434,...,31.622332,256,0,0,"[0.6959084515773997, 0.018004417394784686, 0.0...","[8.912429297011293e-05, 0.000361657941180959, ...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[67493.6395535823, 44382.63569985275, 34585.90...","[33489.50379692876, 15632.94243473894, 11385.0...",9-8770
878,9,8780,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.064148,0.031115,0.000000,0.095263,0.000323,2.517024,...,27.898004,256,1,0,"[0.6934938435739673, 0.026775312387049395, 0.0...","[0.0001479733800862269, 0.0005555139969321071,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[96801.32002238184, 65275.557329208925, 48406....","[52265.10276633037, 24048.047816525235, 15440....",9-8780


In [18]:
df.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

### View best epochs

#### Locations with best chi_sqr

In [19]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
270,3,2700,0.008178,0.077886,0.000000,0.086064,0.000035,5.096999e+00,8.920727e+00,7.664898e+02
664,7,6640,0.084366,0.004462,0.000000,0.088828,0.000515,1.832583e+00,1.521488e+00,1.419854e+02
0,0,0,0.251617,0.004806,0.000000,0.256423,0.003400,3.576025e-01,1.131203e+05,1.361295e+06
206,2,2060,0.017924,0.006395,0.000000,0.024319,0.000070,3.170599e+00,6.628598e+00,6.531434e+02
270,3,2700,0.008178,0.077886,0.000000,0.086064,0.000035,5.096999e+00,8.920727e+00,7.664898e+02
12,0,120,0.209078,0.004806,0.000000,0.213884,0.002236,1.912387e-14,1.486991e+06,1.270652e+09
843,9,8430,0.026290,0.127627,0.000050,0.153967,0.000158,8.838462e+00,3.778679e-01,4.603308e+01
759,8,7590,0.028159,0.141927,0.000024,0.170110,0.000179,5.766723e+00,4.521855e-01,3.277991e+01


In [27]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [20]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [21]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [22]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [26]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [27]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1b',head=8,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=2,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.07625328464221069, 'chi_spec1': 1.2283070570915202}
[0, 9]
{'chi_1': 0.17331106752227488, 'chi_spec1': 8.684712872046113}
[0, 9]
{'chi_1': 0.07625328464221069, 'chi_spec1': 1.2283070570915202}
[2830 2840 3610 4750 5860 6090 6640 7460 7970 8040 8320]


In [28]:
# best_step=[3430,  6460,  7215,  9610, 11725, 14635, 9475]
# best_step=np.arange(40130,40135).astype(int)

In [29]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(11, 22)
[(3, 2830), (3, 2840), (4, 3610), (5, 4750), (6, 5860), (6, 6090), (7, 6640), (8, 7460), (9, 7970), (9, 8040), (9, 8320)]


In [32]:
df_best

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
283,3,2830,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.110747,0.004573,0.000000,0.115320,0.000697,1.116913,...,33.015560,256,0,0,"[0.6949167733387764, 0.02264671322583151, 0.01...","[0.0001039725823105365, 0.0004163744346439183,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[47009.30982154049, 42802.58187150807, 43687.6...","[27016.037394584913, 14687.908222330192, 13123...",3-2830
284,3,2840,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.095859,0.004573,0.000000,0.100432,0.000598,1.640851,...,30.988333,256,0,0,"[0.6945991759493289, 0.02396249767837585, 0.01...","[0.00018920647104323375, 0.0007104759493437207...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[100136.61946798861, 52864.34081464559, 37145....","[61012.442484954394, 27938.568504324125, 20249...",3-2840
361,4,3610,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.062754,0.004470,0.000000,0.067223,0.000402,2.776905,...,32.042693,256,1,0,"[0.6921064620193251, 0.034706067991426164, 0.0...","[0.00011328181574504899, 0.0004652455483660201...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[65223.45858389558, 51359.98418660484, 51522.7...","[31785.162996144256, 15772.156863457714, 12867...",4-3610
475,5,4750,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.083363,0.004695,0.000000,0.088059,0.000392,1.355245,...,30.280487,256,0,0,"[0.6928661589803166, 0.029625177342864552, 0.0...","[0.00018484293636692977, 0.0006920513298823931...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[104448.1146227112, 74314.75176162904, 54247.4...","[48087.603639570894, 26083.86293493029, 19434....",5-4750
586,6,5860,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.041436,0.004573,0.000000,0.046008,0.000204,2.238764,...,32.218006,256,0,0,"[0.69168105174833, 0.033602571441423215, 0.020...","[0.00019559807622077915, 0.000718698368469378,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[108453.64439734444, 70393.82808190699, 50018....","[71228.44158062592, 28382.979617331173, 16648....",6-5860
609,6,6090,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.026061,0.005952,0.000000,0.032013,0.000108,3.125382,...,24.317705,256,0,0,"[0.6921256547149564, 0.031059122043049853, 0.0...","[0.0001627212995142196, 0.0005674585907538136,...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[92442.92069790699, 66578.3204256321, 49750.40...","[45320.39036179166, 22725.99672748429, 15698.3...",6-6090
664,7,6640,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.084366,0.004462,0.000000,0.088828,0.000515,1.832583,...,31.737585,256,0,0,"[0.6946552519991732, 0.02263669964552296, 0.01...","[0.00010919172527783743, 0.0004392530187450763...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[63505.68555607274, 45252.37698462682, 33874.4...","[32469.748307260175, 16873.03201023057, 11281....",7-6640
746,8,7460,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.030175,0.008586,0.000000,0.038761,0.000133,3.295837,...,24.443603,256,0,0,"[0.6927521710575675, 0.028809070547717183, 0.0...","[0.00017347634329753693, 0.0006099022484799929...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[86238.32271494903, 61868.594176543396, 47884....","[47724.6047154363, 22134.405535009882, 16246.7...",8-7460
797,9,7970,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.041123,0.004574,0.000000,0.045697,0.000204,2.650436,...,24.541932,256,0,0,"[0.6935537581629365, 0.026120424234869773, 0.0...","[0.00014051154632926545, 0.0005053457771499486...","[-0.2857142862244898, 0.5476190467616213, 0.71...","[95204.49545348808, 63425.20756536053, 47346.4...","[46344.72998929361, 23951.81665109532, 15391.9...",9-7970
804,9,8040,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.075838,0.004573,0.000000,0

In [33]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
283,3-2830,0.115320,5.589116,33.015560,0.110747,0.004573,0.000000,0.000697,1.116913,688.528300,3,2830
284,3-2840,0.100432,4.590501,30.988333,0.095859,0.004573,0.000000,0.000598,1.640851,365.418664,3,2840
361,4-3610,0.067223,3.261459,32.042693,0.062754,0.004470,0.000000,0.000402,2.776905,424.649690,4,3610
475,5-4750,0.088059,1.650620,30.280487,0.083363,0.004695,0.000000,0.000392,1.355245,107.143561,5,4750
586,6-5860,0.046008,1.951652,32.218006,0.041436,0.004573,0.000000,0.000204,2.238764,174.958560,6,5860
609,6-6090,0.032013,1.161481,24.317705,0.026061,0.005952,0.000000,0.000108,3.125382,149.370180,6,6090
664,7-6640,0.088828,1.521488,31.737585,0.084366,0.004462,0.000000,0.000515,1.832583,141.985414,7,6640
746,8-7460,0.038761,0.409308,24.443603,0.030175,0.008586,0.000000,0.000133,3.295837,43.695240,8,7460
797,9-7970,0.045697,0.475319,24.541932,0.041123,0.004574,0.000000,0.000204,2.650436,56.992356,9,7970
804,9-8040,0.080411,1.092343,31.483343,0.075838,0.004573,0.000000,0.000488,1.908672,81.194273,9,8040


## Plot pixel intensity and spectrum

In [34]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [35]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [36]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('3-2830', '3-2840', '4-3610', '5-4750…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [38]:
df[(df.step>8000)& (df.step<10000)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [ ]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,8025)
# f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

In [ ]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

### Plotting spread of spectrum of input data

In [ ]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [5]:
main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/'
f1=main_dir+'20201208_165658_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a1=np.load(f1)[:,0,:,:]
f2=main_dir+'20201209_055955_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a2=np.load(f2)[:,0,:,:]

print(a1.shape)
print(a2.shape)


(256, 128, 128)
(256, 128, 128)


In [10]:
# f_pixel_intensity(a1)
# f_pixel_intensity(a2)

f_compare_pixel_intensity([a1,a2],label_lst=['img1','img2'],bkgnd_arr=[],log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …